## 3.3 비지도 학습 기반 형태소 분석
- 비지도 학습 기반 형태소 분석: 데이터의 패턴을 모델 스스로 학습하게 함으로써 형태소를 분석하는 방법, 데이터에 자주 등장하는 단어들을 형태소로 인식한다.

### 3.3.1 soynlp 형태소 분석기
- 파이썬 기반 한국어 자연어 처리 패키지
- 비지도 학습 접근법을 지향하므로 하나의 문장 혹은 문서에서보다는 어느 정도 **규모가 있으면서 동질적인 문서 집합(homogeneous documents)**에서 잘 작동한다. 영화 댓글이나 하루치 뉴스 기사 등이 대표적인 사례이다. 
- 단어 점수는 **응집 확률(cohesion probability)**과 **브랜칭 엔트로피(branching entropy)**를 활용한다.
    - 문자열들이 유기적으로 연결돼 함께 자주 나타나고(응집 확률이 높을 때) 그 단어 앞뒤로 다양한 조사, 어미 혹은 다른 단어가 등장하는 경우(브랜칭 엔트로피가 높을 때) 해당 문자열을 형태소로 취급한다. 
    - 예를 들어 말뭉치에서 ``꿀잼``이라는 단어가 연결돼 자주 나타났다면 ``꿀잼``을 형태소라고 본다(응집 확률이 높음)
    - 만일 ``꿀잼`` 앞에 ``영화``, ``정말``, ``너무`` 등 문자열이, 뒤에 ``ㅋㅋ``, ``ㅎㅎ``, ``!!`` 등 패턴이 다양하게 나타났다면 이 역시 ``꿀잼``을 형태소로 취급한다(브랜칭 엔트로피가 높음)
- 비지도 학습이기 때문에 지도 학습과 달리 가지고 있는 말뭉치을 이용하여 **별도의 학습 과정**을 거쳐야 한다. 즉 말뭉치의 분포가 어떻게 돼 있는지 확인하고 단어별로 응집 확률과 브랜칭 엔트로피 점수표를 만드는 절차가 필요하다. 

In [ ]:
# soynlp 단어 점수 학습

from soynlp.word import WordExtractor

corpus_fname = '/notebooks/embedding/data/processed/processed_ratings.txt'
model_fname = '/notebooks/embedding/data/processed/soyword.model'

sentences = [sent.strip() for sent in open(corpus_fname, 'r').readlines()]
word_extractor = WordExtractor(min_frequency=100,
                              min_cohesion_forard=0.05,
                              min_right_branching_entropy=09.0)
word_extractor.train(sentences)
word_extractor.save(model_fname)

In [ ]:
import math
from soynlp.word import WordExtractor
from soynlp.tokenizer import LTokenizer

model_fname = '/notebooks/embedding/data/processed/soyword.model'

word_extractor = WordExtractor(min_frequency=100,
                              min_cohesion_forward=0.05,
                              min_right_branching_entropy=0.0)
word_extractor.load(model_fname)
scores = word_extractor.word_scores()
scores = {key: (scores[key].cohesion_forward * math.exp(scores[key].right_branching_entropy)) for key in scores.keys()}
tokenizer = LTokenizer(scores=scores)
tokens = tokenizer.tokenize('애비는 종이었다')

### 3.3.2 구글 센텐스피스
- 구글에서 공개한 비지도 학습 기반 형태소 분석 패키지
- 바이트 페어 인코딩(BPE, Byte Pair Encoding) 기법 등을 지원
    - BPE의 기본 원리는 말뭉치에서 **가장 많이 등장한 문자열을 병합해 문자열을 함축하는 것**
    - 예를 들어 aaabdaaabac -> ZabdZabac (aa를 Z로 치환) -> ZYdZYac (ab를 Y로 치환), 총 2번의 압축을 진행함
    - 처음 BPE가 쓰인 것은 기계 번역 분야     

In [ ]:
# BPE 학습

import sentencepiece as spm
train = """--input=/notebooks/embedding/data/processed/processed_wiki_ko.txt \
--model_prefix=sentpiece \
--vocab_size=32000 \
--model_type=bpe --character_coverage=0.9995"""
spm.SentencePieceTrainer.Train(train)

In [ ]:
# BPE 어휘 집합 만들기 (bash)
#git pull origin master
#bash preprocess.sh make-bert-vocab

In [ ]:
# BERT FullTokenizer

from bert.tokenization import FullTokenizer

vocab_fname = '/notebooks/embedding/data/processed/bert.vocab'
tokenizer = FullTokenizer(vocab_file=vocab_fname, do_lower_case=False)

tokenizer.tokenize("집에좀 가자")


### 3.3.3 띄어쓰기 교정 

In [ ]:
# soynlp 띄어쓰기 모듈 학습 
from soyspacing.countbase import CountSpace

corpus_fname = '/notebooks/embedding/data/processed/processed_rating.txt'
model_fname = '/notebooks/embedding/data/processed/space-correct.model'

model = CountSpace()
model.train(corpus_fname)
model.save_model(model_fname, json_format=False) 

In [ ]:
# soynlp 띄어쓰기 교정 
from soyspacing.countbase import CountSpace

model_fname = '/notebooks/embedding/data/processed/space-correct.model'
model = CountSpace()
model.load_model(model_fname, json_format=False)
model.correct('어릴때보고 지금다시봐도 재밌어요')


### 3.3.4 형태소 분석 완료된 데이터 다운로드 

In [ ]:
#bash
#git pull origin master
#bash preprocess.sh dump-tokenized

## 3.4 요약
- 임베딩 학습용 말뭉치는 라인 하나가 문서면 좋다.
- **지도 학습 기반의 형태소 분석 모델**은 **언어학 전문가들이 태깅**한 형태소 분석 말뭉치로부터 학습된 기법들이다. 이 모델들은 문자열이 주어졌을 때 사람들이 알려준 정답 패턴에 최대한 가깝게 토크나이즈한다. KoNLPy, Khaiii 등이 이 부류에 속한다.
- **비지도 학습 기반의 형태소 분석 모델**은 데이터의 패턴을 **모델 스스로 학습**하게 함으로써 형태소를 나누는 기법이다. 데이터에 자주 등장한 단어들을 형태소로 인식한다. soynlp, 구글 센턴스피스 등이 이 부류에 속한다.
